In [8]:
import pandas as pd
import re

In [3]:
query = """  select * \nfrom     data limit  1000  """

In [6]:
def queryRevisor(query: str):
    import re

    query = """ select * \nfrom  data limit 1000 """
    result = re.sub(r'\s+', ' ', query)
    return result.strip()


q = queryRevisor(query)

In [11]:
limit = re.findall(r'limit\s+([0-9]+)', q)
limit = limit[0]

In [16]:
from src.query_revisor import Revisor


Revisor(query).run()

ModuleNotFoundError: No module named 'schemas'